<a href="https://colab.research.google.com/github/juliaviolet/BCCOptionsStrategies/blob/main/BCC_Hedging_Strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Hedging Strategies using the Bakshi, Cao and Chen Model**

Author: Julia Morrison

Date: October 2023

**Abstract:**

This project presents an in-depth exploration of various hedging strategies, primarily focusing on the Bakshi, Cao, and Chen (BCC) model for valuing European options under conditions of stochastic volatility and jumps. Starting with model definitions, the notebook introduces the characteristic functions and option pricing techniques associated with the BCC model. Utilizing a dataset of option and underlying asset data, the model's application computes various Greeks, such as delta, vega, gamma, and theta, based on changes in the underlying asset's price. Subsequent sections delve into different hedging strategies, evaluating their effectiveness by calculating daily profits and running balances over a specific timeframe. These strategies range from delta hedging variations to approaches like the ratio spread, married put, and short strangle. Each strategy's performance is assessed by tracking daily profits and balances over a specified duration. Visualizations compare the final balances of each strategy, offering insights into their respective performances.

**Code Overview:**


1. **Model Definitions**:
    - The code starts by defining the Bakshi, Cao, and Chen (BCC) model to value European options under stochastic volatility and jumps.
    - Several functions (`CIR_char_func_nojit`, `H93_char_func_nojit`, `M76_char_func_nojit`, etc.) are defined to compute the characteristic functions and options prices using the BCC model.

2. **Data Import**:
    - The dataset `sorted_updated_merged_data_fixed.csv` containing option and underlying asset data is imported.

3. **Model Application to Data**:
    - The BCC model is applied to the dataset to compute several Greeks (e.g., delta, vega, gamma, theta) and the model prices of the options. The calculations use parameters that were derived in a separate notebook: BCC_Options_Strategies_8. (https://colab.research.google.com/drive/1jxHlUhLEG_aazBbT-k8WgoEMu7MdDgt1?usp=sharing)

    - These Greeks are calculated based on the changes in the underlying asset's price.
    - The computed values are added to the data, and the results are saved to `final_results_fixed.csv`.

4. **Hedging Metrics Calculation**:
    - The code calculates hedging metrics for various strategies such as delta hedging, delta-theta hedging, and shadow delta hedging.

    - The function `calculate_hedging_metrics` computes the daily profit and running balance for each strategy based on the daily hedge values.

    - For each strategy, the running balance is stored, representing the effectiveness of that strategy over the dataset's timeframe.

5. **Additional Hedging Strategies**:
    - The code also evaluates three other hedging strategies:
        - **Ratio Spread**: This strategy identifies the best ratio spread combination that maximizes the profit for both call and put options.
        - **Married Put**: This strategy evaluates the profitability of buying a put option as insurance for a stock position.
        - **Short Strangle**: This strategy calculates the profit or loss from selling out-of-the-money call and put options.

6. **Visualization and Results**:
    - The end balances of all strategies are aggregated and visualized in a bar chart using the Plotly library.
    - The chart provides a clear comparison of the effectiveness of each strategy.
    - The final balances for each strategy are printed out.
    - The running balances of the delta hedging strategies are visualized in a line chart.
   
The code provides an analysis of various hedging strategies applied to a dataset of options and an underlying asset. The BCC model is used to compute option prices and their Greeks, and the results are visualized to compare the effectiveness of each strategy.

### **Code Overview:** `BCC_model_greeks.py`

**1. Model Definitions:**
      The Bakshi, Cao, and Chen (BCC) model is employed to value European options in scenarios characterized by stochastic volatility and jumps.

 **Characteristic Functions**:
       `CIR_char_func_nojit`: Computes the characteristic function for the Cox-Ingersoll-Ross (CIR) interest rate model. This model captures the dynamics of interest rate movements.
      `H93_char_func_nojit`: Computes the characteristic function for the Heston '93 (H93) model. The H93 model is a stochastic volatility model that captures the volatility's randomness in option pricing.
       `M76_char_func_nojit`: Computes the characteristic function for the Merton '76 (M76) model. This model introduces jumps in the return process, accounting for sudden and significant price changes.
       `BCC_char_func_nojit`: Represents the Bakshi, Cao, and Chen (BCC) model's characteristic function. It integrates the CIR, H93, and M76 models to provide a more comprehensive model that values European options, considering both stochastic volatility and jumps.

   **Option Pricing**:
      - `BCC_call_value_nojit`: Utilizes the combined BCC characteristic function to evaluate the price of European call options. This function integrates over the characteristic function to derive the options' prices under the BCC model framework.

**2. Data Import:**
    - The dataset, named `sorted_updated_merged_data_fixed.csv`, which contains details about the options and the underlying asset, is imported.

**3. Model Application to Data:**
    - The BCC model is applied to the dataset for the computation of various Greeks, namely delta, vega, gamma, and theta. Additionally, the model's price for the option is determined. For these computations, parameters sourced from a separate notebook titled [BCC_Options_Strategies_8](https://colab.research.google.com/drive/1jxHlUhLEG_aazBbT-k8WgoEMu7MdDgt1?usp=sharing) are utilized.

**4.  Greeks Calculations**:
       `BCC_delta_nojit`: Sensitivity of the option price to alterations in the underlying asset's price (delta).
       `BCC_vega_nojit`: Sensitivity of the option price to volatility changes (vega).
       `BCC_theta_nojit`: Sensitivity of the option price to the progression of time (theta).
       `BCC_gamma_nojit`: Rate at which Delta changes with respect to modifications in the underlying price (gamma).
     Following the calculations, the derived values are amalgamated with the data. The consolidated results are stored in a file named `final_results_fixed.csv`.

In [62]:
import numpy as np
import pandas as pd
from BCC_model_greeks_PEP8 import *
from scipy.integrate import quad

# read the final_merged_data CSV file
merged_data = pd.read_csv('sorted_updated_merged_data_fixed.csv')

# calculate the price change and its absolute value for the underlying asset
merged_data['PRICE_CHANGE'] = merged_data['CF_CLOSE_y'].diff()
merged_data['PRICE_CHANGE_ABS'] = merged_data['PRICE_CHANGE'].abs()

# Define the initial parameters
initial_params = [3.76807610e+00, 1.00000000e-03, 8.77986615e-03, 5.67355018e-03,
 4.16586532e+00, 2.29651712e-03, 1.14133804e-02, 5.03063522e-01,
 0.00000000e+00, 3.95979221e-01, 7.10854746e-02, 4.01539560e-02,
 6.99623759e-02, 9.99763692e-02, 5.00002718e-01]

# Refactor the repetitive part using a loop
functions = [
    (BCC_option_value_nojit, 'MODEL_PRICE', ['PUTCALLIND']),
    (BCC_delta_nojit, 'DELTA', ['PRICE_CHANGE_ABS', 'PUTCALLIND']),
    (BCC_vega_nojit, 'VEGA', ['PRICE_CHANGE_ABS', 'PUTCALLIND']),
    (BCC_theta_nojit, 'THETA', ['PUTCALLIND']),
    (BCC_gamma_nojit, 'GAMMA', ['PRICE_CHANGE_ABS', 'PUTCALLIND'])
]

def generate_args(row, func_args):
    base_args = [
        10337.02,
        row['STRIKE_PRC'],
        row['T'],
        *initial_params[3:12],
        initial_params[0],
        initial_params[1],
        initial_params[2],
        *initial_params[12:]
    ]

    for arg in func_args:
        if arg in row:
            base_args.append(row[arg])

    return base_args

for func, column_name, additional_args in functions:
    merged_data[column_name] = merged_data.apply(lambda row: func(*generate_args(row, additional_args)), axis=1)

# calculate hedge amounts based on the Greeks and the price change
def calculate_hedge_amounts(df):
    df['DELTA_HEDGE'] = df['DELTA'] * df['PRICE_CHANGE']
    df['VEGA_HEDGE'] = df['VEGA'] * df['PRICE_CHANGE']
    df['THETA_HEDGE'] = df['THETA'] * df['PRICE_CHANGE']
    df['GAMMA_HEDGE'] = df['GAMMA'] * df['PRICE_CHANGE']

calculate_hedge_amounts(merged_data)

# extract relevant columns for final results
final_columns = ['Unnamed: 0', 'Instrument', 'CF_DATE', 'EXPIR_DATE',
                 'PUTCALLIND', 'STRIKE_PRC', 'CF_CLOSE_x', 'IMP_VOLT',
                 'CF_CLOSE_y',
                 'DELTA', 'VEGA', 'THETA', 'GAMMA', 'DELTA_HEDGE', 'VEGA_HEDGE',
                 'THETA_HEDGE', 'GAMMA_HEDGE']

final_df = merged_data[final_columns]

# save to a new CSV file
final_df.to_csv('final_results_2.csv', index=False)


/content/BCC_model_greeks_PEP8.py:83: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

/content/BCC_model_greeks_PEP8.py:83: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.



Pandas apply with lambda: Using the apply method with lambda functions can sometimes lead to minor discrepancies due to the way operations are vectorized. This is due to the underlying intricacies of how operations are performed, vectorized, and optimized. In this case, the discrepancies did not produce different final balances for any of the strategies. (see apply_lambda_note.ipynb https://colab.research.google.com/drive/1KQqFTkmrHDUP9FhdbVPLcPQqhB0W5mdP?usp=sharing)

In [63]:
import plotly.graph_objects as go

# filter the data for call and put options
call_data = merged_data[merged_data['PUTCALLIND'] == 'CALL']
put_data = merged_data[merged_data['PUTCALLIND'] == 'PUT ']

# function to create the figure for a given data subset for time
def create_figure(data, title):
    fig = go.Figure()

    # hovertemplate for Model Price
    hovertemplate_model = ('<span style="color:royalblue"><b>Model Price</b>: %{y:.4f}</span><br>'
                           '<span style="color:red"><b>Delta</b>: %{customdata[1]:.4f}</span><br>'
                           '<span style="color:green"><b>Vega</b>: %{customdata[2]:.4f}</span><br>'
                           '<span style="color:orange"><b>Theta</b>: %{customdata[3]:.4f}</span><br>'
                           '<span style="color:purple"><b>Gamma</b>: %{customdata[4]:.4f}</span><br>'
                           '<extra></extra>')

    # hovertemplate for Underlying Asset
    hovertemplate_asset = ('<span style="color:gray"><b>Underlying Asset</b>: %{y:.4f}</span><br>'
                           '<extra></extra>')

    # model price trace
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['MODEL_PRICE'],
                             mode='lines',
                             name='Model Price',
                             line=dict(color='royalblue'),
                             customdata=data[['MODEL_PRICE', 'DELTA', 'VEGA', 'THETA', 'GAMMA']].values,
                             hovertemplate=hovertemplate_model))

    # underlying asset trace
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['CF_CLOSE_y'],
                             mode='lines',
                             name='Underlying Asset',
                             line=dict(color='gray'),
                             hovertemplate=hovertemplate_asset))

    # greeks (with hoverinfo set to 'skip')
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['DELTA'],
                             mode='lines',
                             name='Delta',
                             line=dict(color='red'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['VEGA'],
                             mode='lines',
                             name='Vega',
                             line=dict(color='green'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['THETA'],
                             mode='lines',
                             name='Theta',
                             line=dict(color='orange'),
                             hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=data['CF_DATE'], y=data['GAMMA'],
                             mode='lines',
                             name='Gamma',
                             line=dict(color='purple'),
                             hoverinfo='skip'))

    # update the layout
    fig.update_layout(title=title,
                      xaxis_title='Date',
                      yaxis_title='Value',
                      template='plotly',
                      hovermode='x unified')
    fig.show()


# Create figures for call and put options over time
create_figure(call_data, 'Call Options: Model, Underlying Asset, and Greeks Over Time')
create_figure(put_data, 'Put Options: Model, Underlying Asset, and Greeks Over Time')


**Delta (Δ)**:
Delta represents the sensitivity of an option's price to a change in the price of the underlying asset. In the provided code, delta is computed by evaluating the option price for positive and negative changes in the underlying asset's price. In the plot, the delta line gives an idea of how sensitive the option's price is to movements in the underlying asset's price over time.

**Vega (ν)**:
Vega measures the sensitivity of the option's price to changes in volatility. The code calculates vega by evaluating the option price for small increases and decreases in the volatility parameter. The vega line in the plot highlights how sensitive the option's price is to volatility changes over different timeframes.

**Gamma (Γ)**:
Gamma provides insight into the rate of change of delta with respect to changes in the underlying asset's price. It is essentially the second derivative of the option price concerning the underlying price. The code computes gamma by evaluating delta for tiny positive and negative changes in the underlying asset's price. The gamma line in the plot showcases how this sensitivity evolves over time.

**Theta (Θ)**:
Theta gauges the sensitivity of the option's price to the passage of time, representing time decay. In the code, theta is computed by evaluating the option price for small advancements and retractions in time. The theta line in the plot offers a visual representation of how the option's price is impacted by time decay.

In [64]:
import pandas as pd
import plotly.graph_objects as go

# load the data
data = pd.read_csv('final_results_2.csv')


# filter the data for call and put options
call_data = data[data['PUTCALLIND'] == 'CALL']
put_data = data[data['PUTCALLIND'] == 'PUT ']


# create a plotting function for each Greek
def plot_greek(greek_name):
    fig = go.Figure()

    # define hovertemplate to show the 'Unnamed: 0' value, the date, and the greek value
    hovertemplate = "<b>Date</b>: %{customdata[1]}<br><b>" + greek_name + "</b>: %{y:.4f}<br><b>Option Row Number</b>: %{customdata[0]}<extra></extra>"

    # add traces for Call and Put options
    fig.add_trace(go.Scatter(x=call_data['CF_DATE'],
                             y=call_data[greek_name],
                             customdata=call_data[['Unnamed: 0', 'CF_DATE']],
                             mode='lines',
                             name=f'CALL {greek_name}',
                             hovertemplate=hovertemplate))

    fig.add_trace(go.Scatter(x=put_data['CF_DATE'],
                             y=put_data[greek_name],
                             customdata=put_data[['Unnamed: 0', 'CF_DATE']],
                             mode='lines',
                             name=f'PUT {greek_name}',
                             hovertemplate=hovertemplate))

    # Update the layout
    fig.update_layout(title=f'{greek_name} Over Time for CALL and PUT Options',
                      xaxis_title='Date',
                      yaxis_title=greek_name,
                      template='plotly')
    fig.show()


# generate the plots
plot_greek('DELTA')
plot_greek('VEGA')
plot_greek('THETA')
plot_greek('GAMMA')


**Delta (Δ):**
Delta represents the rate of change of the option's price with respect to changes in the underlying asset's price. Observations from the plot might show that delta remains relatively stable during certain periods and displays significant fluctuations during others. A Delta close to 1 for call options or -1 for put options indicates deep in-the-money options, signifying that the option's price moves almost in tandem with the underlying asset. On the other hand, Delta values near 0 suggest out-of-the-money options, where the option's price is less sensitive to changes in the underlying asset.

**Vega (ν):**
Vega gauges the sensitivity of the option's price to changes in volatility. A high vega for both call and put options suggests that the market anticipates notable future volatility. This means that the options' prices will be more sensitive to shifts in implied volatility. Hence, if the vega is observed to be high, it could indicate market anticipation of significant price movements in the underlying asset.

**Theta (Θ):**
Theta measures the sensitivity of the option's price to the passage of time. Long option positions (both calls and puts) usually have negative theta, meaning they lose value over time. Short positions typically have positive theta, benefiting from time decay. Observing an increasing magnitude in theta over time might suggest that time decay becomes more pronounced as options near their expiration dates.

**Gamma (Γ):**
Gamma indicates the rate of change in delta concerning changes in the underlying asset's price. If gamma is relatively stable, it suggests a consistent sensitivity of Delta to price changes. A lower gamma for options that are either deep in-the-money or out-of-the-money means that the delta of these options is relatively stable. This is because, for such options, delta is nearing its maximum or minimum bounds (1 or 0 for calls and -1 or 0 for puts), and thus changes less with price movements in the underlying asset.

In [65]:
final_results_data = pd.read_csv('final_results_2.csv')

# add the missing 'Price_Change' and 'Price_Change_Abs' columns to the DataFrame
final_results_data['Price_Change'] = final_results_data['CF_CLOSE_y'].diff()
final_results_data['Price_Change_Abs'] = final_results_data['Price_Change'].abs()

# show the first few rows to confirm that the columns have been added
final_results_data.head()

# save the DataFrame to a new CSV file
final_results_data.to_csv('final_results_price_2.csv', index=False)

In [66]:
# load the newly provided dataset
final_results_fixed_price = pd.read_csv('final_results_price_2.csv')


# round the hedge values and create new columns
for hedge_type in ['DELTA', 'VEGA', 'THETA', 'GAMMA']:
    final_results_fixed_price[f'{hedge_type}_HEDGE_ROUND'] = final_results_fixed_price[f'{hedge_type}_HEDGE'].round().astype(float)


# reorder the columns to place the rounded columns to the right of their respective original columns
reordered_columns = list(final_results_fixed_price.columns[:-4])
for hedge_type in ['DELTA', 'VEGA', 'THETA', 'GAMMA']:
    reordered_columns.append(f'{hedge_type}_HEDGE')
    reordered_columns.append(f'{hedge_type}_HEDGE_ROUND')


final_results_rounded = final_results_fixed_price[reordered_columns]
final_results_rounded.head()

final_results_rounded.to_csv('final_results_price_rounded_2.csv', index=False)

In [67]:
import pandas as pd

# load the data
final_results_data = pd.read_csv('final_results_price_rounded_2.csv')

# remove the first row to avoid NaN in 'Price_Change' and 'Price_Change_Abs'
final_results_data = final_results_data.iloc[1:].reset_index(drop=True)

# initialize variables
initial_portfolio_value = 10000.00  # Starting with a portfolio value of $10,000
transaction_cost_per_contract = 0.75  # Transaction cost of $0.75 per contract

# initialize running balance for combined strategy
combined_running_balance = initial_portfolio_value

# initialize DataFrame to store metrics for combined strategy
combined_hedging_metrics = pd.DataFrame()


# loop through each row in the dataset
for index, row in final_results_data.iterrows():
    metrics_row = row.to_dict()  # Start with existing row data
    daily_profit = 0.0  # Initialize daily profit for combined strategy

    # check profitability for each strategy and update running balance if profitable
    for strategy in ['DELTA', 'VEGA', 'THETA', 'GAMMA']:
        hedge_col = f"{strategy}_HEDGE_ROUND"
        transaction_value = -1 * row[hedge_col] * row['CF_CLOSE_x']
        daily_buy_transaction_cost = abs(transaction_value) * transaction_cost_per_contract if transaction_value > 0 else 0
        daily_sell_transaction_cost = abs(transaction_value) * transaction_cost_per_contract if transaction_value < 0 else 0
        total_cost_revenue = transaction_value - daily_buy_transaction_cost - daily_sell_transaction_cost

        if total_cost_revenue > 0:  # if the hedge is profitable
            daily_profit += total_cost_revenue  # Add to daily profit
            metrics_row[f"{strategy}_Transaction_Value"] = transaction_value
            metrics_row[f"{strategy}_Daily_Buy_Transaction_Cost"] = daily_buy_transaction_cost
            metrics_row[f"{strategy}_Daily_Sell_Transaction_Cost"] = daily_sell_transaction_cost
            metrics_row[f"{strategy}_Total_Cost/Revenue"] = total_cost_revenue
        else:  # if the hedge is not profitable
            metrics_row[f"{strategy}_Transaction_Value"] = 0
            metrics_row[f"{strategy}_Daily_Buy_Transaction_Cost"] = 0
            metrics_row[f"{strategy}_Daily_Sell_Transaction_Cost"] = 0
            metrics_row[f"{strategy}_Total_Cost/Revenue"] = 0

    # update the running balance for the combined strategy
    combined_running_balance += daily_profit
    metrics_row["Combined_Running_Balance"] = combined_running_balance

    # append metrics for this row to the combined DataFrame
    combined_hedging_metrics = pd.concat([combined_hedging_metrics, pd.DataFrame([metrics_row])], ignore_index=True)

# save the DataFrame to a new CSV file
combined_hedging_metrics.to_csv('combined_hedging_metrics_rounded_2.csv', index=False)


**Hedging Strategies Overview**

Each strategy calculates daily profits, adjusting for transaction costs and dynamic market conditions:

1. **Delta Theta**: Incorporates delta, vega, theta, and gamma.
2. **Delta**: Focuses on delta, vega, and gamma, excluding theta.
3. **Shadow Delta**: Adjusts delta based on daily price and volatility changes, and includes theta.
4. **Shadow Delta Without Theta**: An iteration of the shadow delta strategy without the theta component.
5. **Adjusted Shadow Delta**: Modifies the delta with a volatility smile factor, reflecting the difference between the option's strike price and the actual stock price.
6. **Adjusted Shadow Delta Without Theta**: A variant of the adjusted shadow delta that omits theta.
7. **Taleb's Adjusted Shadow Delta**: Uses the discrete measurement of gamma hedge as introduced in the linked article, adjusting the delta hedge with a gamma adjustment which is a function of the square of the price change.
8. **Taleb's Adjusted Shadow Delta Without Theta**: Similar to Taleb's Adjusted Shadow Delta but excludes the theta component for the adjustment.
9. **Ratio Spread**: Seeks the optimal combination of options to maximize profits.
10. **Married Put**: Involves buying a put option for a stock that's already owned to provide protection against potential stock price declines.
11. **Short Strangle**: Sells out-of-the-money call and put options, aiming to profit when the underlying asset's price remains between the two strike prices.


A note on strategies 7 and 8:

The provided article (linked below) discusses the concept of shadow gamma. It refers to the discrete measurement of the gamma hedge in the context of hedging options. The article suggests a methodology to calculate the shadow gamma by adjusting the delta hedge using a gamma adjustment, which is a function of the square of the price change. In the provided Python code, we see the implementation of various hedging strategies, including the adjusted shadow delta approach, which closely aligns with the article's content. The function `compute_taleb_adjusted_shadow_delta` captures this approach, wherein the delta hedge is adjusted using the gamma hedge and the square of the price change. This adjustment is used to determine the impact of the option's second-order Greek (gamma) on the delta hedge.

https://financetrainingcourse.com/education/2014/06/calculating-shadow-gamma-talebs-approach-for-the-second-order-option-greek/#:~:text=In%20this%20discrete%20measurement%2C%20gamma,down%20by%20an%20incremental%20value.

In [68]:
%run all_hedging_strategies_PEP8.py

Adjusted Shadow Delta: $289,865,214.32
Shadow Delta: $280,969,226.40
Taleb Shadow Delta: $280,969,017.93
Delta Theta: $280,967,607.15
Married Put: $23,812,309.20
Short Strangle: $18,139,150.00
Adjusted Shadow Delta Without Theta: $14,383,843.95
Delta: $4,663,241.85
Taleb Shadow Delta Without Theta: $4,634,338.80
Shadow Delta Without Theta: $4,628,341.78
Ratio Spread (Calls): $832,329.90
Ratio Spread (Puts): $810,250.20


In [69]:
import plotly.graph_objects as go

# extract the 'Running_Balance' columns and the 'CF_DATE' column for each strategy
dates = delta_theta_metrics['CF_DATE']  # Assuming 'CF_DATE' is the date column

# store the metrics dataframes and their names in lists for including and excluding theta
metrics_including_theta = [
    ("Delta Theta", delta_theta_metrics),
    ("Shadow Delta", shadow_delta_metrics),
    ("Adjusted Shadow Delta", adjusted_shadow_delta_metrics),
    ("Taleb Shadow Delta", taleb_shadow_delta_metrics)
]

metrics_excluding_theta = [
    ("Delta", delta_metrics),
    ("Shadow Delta Without Theta", shadow_delta_without_theta_metrics),
    ("Adjusted Shadow Delta Without Theta", adjusted_shadow_delta_without_theta_metrics),
    ("Taleb Shadow Delta Without Theta", taleb_shadow_delta_without_theta_metrics)
]

# sort these lists by the final balance of each dataframe in descending order
metrics_including_theta_sorted = sorted(metrics_including_theta, key=lambda x: x[1]['Running_Balance'].iloc[-1], reverse=True)
metrics_excluding_theta_sorted = sorted(metrics_excluding_theta, key=lambda x: x[1]['Running_Balance'].iloc[-1], reverse=True)


# create plots using the sorted lists
def create_figure(metrics_sorted, title):
    plot_data = [go.Scatter(x=dates,
                            y=metric_df['Running_Balance'],
                            mode='lines',
                            name=name,
                            hovertemplate='<b>%{fullData.name} Balance:</b> %{y:$,.2f}<extra></extra>') for name, metric_df in metrics_sorted]
    fig = go.Figure(data=plot_data)
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Running Balance",
        template="plotly",
        hovermode='x unified'
    )
    fig.show()

create_figure(metrics_including_theta_sorted, "Running Balance Over Time (Including Theta)")
create_figure(metrics_excluding_theta_sorted, "Running Balance Over Time (Excluding Theta)")


**Conclusion:**

The hedging strategies outlined in this project offer a toolkit for managing portfolio risks in the options market. By leveraging the BCC model's capabilities, it has computed the Greeks, which play an instrumental role in these strategies. The final visualizations display a clear comparative view of each strategy's effectiveness over time. Each strategy has its own set of advantages and use-cases, helping investors make informed decisions based on varying market conditions and risk appetites.